<a href="https://colab.research.google.com/github/zuzann18/Loan_default/blob/main/Applied_Data_Science_Loan_Default_Zuzanna_Walus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loan Default Prediction**


## **Problem Definition**

### **The Context: Why is this Problem Important?**

Loan default is a major challenge in retail banking, directly affecting profitability and risk exposure. Manual, judgment-based credit assessment processes are prone to inefficiency and bias. Automating this process using data-driven methods reduces human error, ensures regulatory compliance (e.g., Equal Credit Opportunity Act), and provides fairer, more transparent lending decisions.
Additional business view:
Beyond loss mitigation, advanced risk prediction enables better portfolio management, dynamic product pricing, and enhanced customer experience, all of which contribute to competitive advantage.

#### **Additional Business Perspective**
From a strategic point of view, automating and optimizing credit risk evaluation brings:
- **Operational efficiency** – reducing processing time and manpower requirements.
- **Improved risk management** – avoiding bad loans improves balance sheet health.
- **Regulatory alignment** – adhering to frameworks like the Equal Credit Opportunity Act ensures fair lending practices.
- **Competitive advantage** – faster and more accurate decisions enhance customer experience and loyalty.

### **The Objectives: What is the Goal?**

The primary goal is to build a binary classification model that predicts whether a loan applicant is likely to default (BAD = 1) or repay (BAD = 0), using the features available at the time of loan application.

#### **Extended Objective View**
- Proactively **identify high-risk applicants** before loan disbursal.
- Offer **interpretable and explainable** decisions to justify loan denials.
- Generate **feature-based recommendations** for refining the bank's approval policy.
- Aid in **bias mitigation** by ensuring that historical discrimination does not propagate through the model.

### **The Key Questions**

- Which variables are most predictive of loan default?
- What is the best performing model (accuracy vs interpretability) for this problem?
- How can the model be validated for fairness and generalizability?
- What steps should the bank take based on model insights?
- Can we balance business objectives (low false negatives) with regulatory fairness?

### **The Problem Formulation: Data Science View**

This problem is a supervised binary classification task. Given historical data on loan applications and their outcomes, we aim to:

- Train a model to predict the `BAD` label based on input features like income, employment status, existing debts, property value, and more.
- **Focus on predicting `BAD = 1` (loan default), since the business impact of lending to defaulters is high.**
- **Minimize False Negatives (i.e., cases where a defaulter is incorrectly classified as non-defaulter), which represent the costliest errors.**
- Evaluate models using classification metrics (e.g., ROC-AUC, Recall, Precision-Recall Curve, F1-Score) appropriate for imbalanced datasets.
- Emphasize model interpretability (e.g., using logistic regression, decision trees, SHAP) to support business deployment.


## **Data Description:**
The Home Equity dataset (HMEQ) contains baseline and loan performance information for 5,960 recent home equity loans. The target (BAD) is a binary variable that indicates whether an applicant has ultimately defaulted or has been severely delinquent. This adverse outcome occurred in 1,189 cases (20 percent). 12 input variables were registered for each applicant.


* **BAD:** 1 = Client defaulted on loan, 0 = loan repaid

* **LOAN:** Amount of loan approved.

* **MORTDUE:** Amount due on the existing mortgage.

* **VALUE:** Current value of the property.

* **REASON:** Reason for the loan request. (HomeImp = home improvement, DebtCon= debt consolidation which means taking out a new loan to pay off other liabilities and consumer debts)

* **JOB:** The type of job that loan applicant has such as manager, self, etc.

* **YOJ:** Years at present job.

* **DEROG:** Number of major derogatory reports (which indicates a serious delinquency or late payments).

* **DELINQ:** Number of delinquent credit lines (a line of credit becomes delinquent when a borrower does not make the minimum required payments 30 to 60 days past the day on which the payments were due).

* **CLAGE:** Age of the oldest credit line in months.

* **NINQ:** Number of recent credit inquiries.

* **CLNO:** Number of existing credit lines.

* **DEBTINC:** Debt-to-income ratio (all your monthly debt payments divided by your gross monthly income. This number is one way lenders measure your ability to manage the monthly payments to repay the money you plan to borrow.

## **Import the necessary libraries and Data**

In [1]:
!pip install pingouin

# Import necessary libraries
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
)
from sklearn.pipeline import Pipeline as SklearnPipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer, classification_report, confusion_matrix
)
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
import scipy.stats as st
from mlxtend.evaluate import mcnemar_table, mcnemar, cochrans_q
import pingouin as pg
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

import joblib
import shap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 8.7 MB/s eta 0:00:00


In [ ]:

from google.colab import files
import io

uploaded = files.upload()

# Load dataset
#df = pd.read_csv('hmeq.csv')
# Get the actual filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[filename]))
df.head()




# Data overview: shape, types, nulls, duplicates


In [ ]:

# Load dataset
df = pd.read_csv('hmeq.csv')
df.head()


In [ ]:

# Data overview: shape, types, nulls, duplicates
print("Shape of dataset:", df.shape)
print("\nData Types:")
print(df.dtypes)



In [ ]:
# Select numeric and categorical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()

# Separate count-type vs continuous numeric features
count_like_vars = []
continuous_vars = []

for col in numeric_cols:
    unique_vals = df[col].dropna().unique()
    if pd.api.types.is_integer_dtype(df[col]) and df[col].nunique() < 20:
        count_like_vars.append(col)
    elif col != 'BAD':  # Exclude target
        continuous_vars.append(col)

# Define target variable
target_var = 'BAD' if 'BAD' in df.columns else None

# Output results
print("Detected Feature Types:")
print(f"- Continuous numeric: {continuous_vars}")
print(f"- Count (discrete integers): {count_like_vars}")
print(f"- Categorical: {categorical_cols}")
if target_var:
    print(f"- Target variable: {target_var}")

In [ ]:
print("\nMissing Values Check:")
missing_values = df.isnull().sum()
print(missing_values)

print("\nNumber of Duplicate Rows:")
print(df.duplicated().sum())



In [ ]:
def format_number(x):
    """
    Format a numeric value to 3 decimal places, but if the value is zero (or nearly zero)
    then return '0' (as string). Also remove unnecessary trailing zeros.
    """
    try:
        num = float(x)
        # If the absolute value is negligibly small, print 0
        if abs(num) < 1e-7:
            return "0"
        # Format with three decimals first
        s = f"{num:.3f}"
        # Remove trailing zeros and a trailing dot if any
        s = s.rstrip('0').rstrip('.') if '.' in s else s
        return s
    except Exception:
        return x

# Example: Calculate and format the missing_percentage series from the notebook
missing_percentage = (df.isnull().sum() / len(df)) * 100
formatted_missing_percentage = missing_percentage.apply(format_number)
print("Missing percentage per column %:")
print(formatted_missing_percentage)





In [ ]:
missing_like = ['NA', 'N/A', 'na', 'n/a', 'null', 'NULL', '', ' ', None]

for col in df.columns:
    missing_count = df[col].isin(missing_like).sum()
    if missing_count > 0:
        print(f"{col} has {missing_count} suspicious missing-like values.")

In [ ]:
for col in df.select_dtypes(include='object'):
    stripped = df[col].astype(str).str.strip()
    print(f"{col}: {(stripped == '').sum()} empty strings after strip.")

In [ ]:
print("\nUnique Values per Column:")
print(df.nunique())

## Summary Statistics

In [ ]:

# Summary statistics
summary = df.describe().round(3)
summary = summary.applymap(
    lambda x: f"{x:.3f}".rstrip("0").rstrip(".")
    if isinstance(x, float) else x
)

print("\nStatistical Summary:\n", summary)



## Observations Summary Statistics

**Correlation Insights:**
- **DELINQ** (delinquent credit lines) shows a moderate positive correlation (0.354) with **BAD** (loan default), indicating a higher number of delinquencies may be associated with defaults.
- **DEROG** (major derogatory reports) correlates positively (0.276) with **BAD**, suggesting its potential as a risk indicator.
- Other features like **LOAN**, **MORTDUE**, and **VALUE** are strongly inter-correlated, which may imply multicollinearity issues that need attention during modeling.

**Data Quality and Missing Values:**
- The main DataFrame has complete records for loan performance and applicant details, but the *outliers* subset reveals missing values in fields like **DEBTINC** and some categorical variables.
- Missing observations in variables such as **MORTDUE**, **VALUE**, **JOB**, etc., are evident in the *outliers* DataFrame, which might affect model training if not properly imputed.

**Outlier Detection:**
- The *outliers* DataFrame contains 1201 records, emphasizing the need for careful treatment of extreme values, especially in features like **DELINQ**, **DEROG**, and **DEBTINC**.

**General Observations:**
- The dataset contains 5960 records with 13 attributes, combining both numerical and categorical data.
- Attention should be paid to handling multicollinearity and the imputation of missing values to ensure robust model performance.

## Treating Missing  - Imputation

- In this section  
1) IterativeImputer (MICE)
2) Random sampling

### Enable the experimental IterativeImputer (MICE)

In [ ]:
# Enable the experimental IterativeImputer (MICE)
# Identify numeric columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Apply iterative imputation (MICE) to numeric features
mice_imputer = IterativeImputer(random_state=42)
df[num_cols] = mice_imputer.fit_transform(df[num_cols])

# Check that missing values have been imputed
print("Missing values after MICE imputation:")
print(df[num_cols].isnull().sum())

### Handling Missing Values for Nominal Unordered Variables (JOB, REASON) - Random Sampling

Statistical operations based on ordering (such as mean or median) are inappropriate and meaningless for such data types.

#### Indexes of Nans for REASON before imputation

In [ ]:
nan_list = df[df['REASON'].isna()].index.tolist()
print(nan_list)

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='REASON', data=df)
plt.title('Distribution of REASON before Randomized Imputation')
plt.show()

#### Random Sampling REASON

In [ ]:
probs = df['REASON'].value_counts(normalize=True)
mask_nan = df['REASON'].isna()
n_nan = mask_nan.sum()
draws = np.random.choice(probs.index, size=n_nan, p=probs.values)
df.loc[mask_nan, 'REASON'] = draws

This method:

1)Preserves **original proportions** between DebtCon and HomeImp.

2)**Avoids introducing bias** towards a particular category.

3)Maintains data realism, important for model performance and interpretability.

#### Nans of REASON after imputation

In [ ]:
nan_list = df[df['REASON'].isna()].index.tolist()
print(nan_list)

In [ ]:
# Plotting REASON distribution after imputation

plt.figure(figsize=(6,4))
sns.countplot(x='REASON', data=df)
plt.title('Distribution of REASON after Randomized Imputation')
plt.show()


In [ ]:
# Calculate the probability distribution of existing non-missing JOB values
probs = df['JOB'].value_counts(normalize=True)

# Identify missing values
mask_nan = df['JOB'].isna()
n_nan = mask_nan.sum()

# Randomly draw replacements based on the observed distribution
draws = np.random.choice(probs.index, size=n_nan, p=probs.values)

# Fill missing values with the drawn random categories
df.loc[mask_nan, 'JOB'] = draws

#### Why Use Random Sampling Instead of Mode?
Mode-only imputation could overpopulate the most common category (e.g., "Other"), artificially simplifying the job distribution.

Random sampling preserves the natural variability of occupations.

Especially important in credit scoring: different job types may have different risk profiles (e.g., self-employed vs salaried employees).

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='JOB', data=df)
plt.title('Distribution of JOB after Randomized Imputation')
plt.xticks(rotation=45)
plt.show()

## **Exploratory Data Analysis (EDA) and Visualization**

### **Univariate Analysis**

#### Checking Outliers

In [ ]:
def plot_continuous_outliers(df, column):
    """
    Plots a histogram with KDE and a boxplot for a continuous numeric column,
    using IQR-based outlier thresholds. It also prints the count and percentage
    of outliers below and above the bounds (with a '%' sign).

    Parameters:
        df (DataFrame): The input DataFrame.
        column (str): The name of the continuous numeric column to analyze.
    """
    data = df[column].dropna()
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Plot histogram with KDE and add vertical lines for lower and upper bounds.
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    sns.histplot(data, bins=20, kde=True, color='skyblue', ax=axes[0])
    axes[0].axvline(lower_bound, color='red', linestyle='--', label='Lower Bound')
    axes[0].axvline(upper_bound, color='green', linestyle='--', label='Upper Bound')
    axes[0].set_title(f"Histogram and KDE of {column}")
    axes[0].legend()

    # Plot boxplot with outlier boundaries
    sns.boxplot(x=data, color='steelblue', ax=axes[1])
    axes[1].axvline(lower_bound, color='red', linestyle='--')
    axes[1].axvline(upper_bound, color='green', linestyle='--')
    axes[1].set_title(f"Boxplot of {column}")

    plt.tight_layout()
    plt.show()

    total_count = len(data)
    outliers_below = (data < lower_bound).sum()
    outliers_above = (data > upper_bound).sum()
    percentage_below = (outliers_below / total_count) * 100
    percentage_above = (outliers_above / total_count) * 100

    print(f"\n{column} IQR Outlier Boundaries:")
    print(f" - Lower Bound: {lower_bound:.2f}")
    print(f" - Upper Bound: {upper_bound:.2f}")
    print(f" - Outliers below lower bound: {outliers_below} ({percentage_below:.2f}%)")
    print(f" - Outliers above upper bound: {outliers_above} ({percentage_above:.2f}%)")

### Boxplot: DEROG Distribution by Loan Default (BAD)

This boxplot compares the distribution of `DEROG` (major derogatory credit reports) between non-defaulters (`BAD = 0`) and defaulters (`BAD = 1`).

**Key observations:**
- Applicants who defaulted (`BAD = 1`) tend to have significantly higher values of `DEROG`, with many outliers above 2 and some reaching as high as 10.
- In contrast, most non-defaulters (`BAD = 0`) have zero or near-zero `DEROG` values.
- The difference in distribution between the two groups reinforces the importance of `DEROG` as a risk factor for loan default prediction.

This insight supports the inclusion of `DEROG` in the final model and may also justify a binning or transformation strategy to improve performance in linear models.


In [ ]:
# Compare DEROG distribution between default status groups
plt.figure(figsize=(8, 5))
sns.boxplot(x='BAD', y='DEROG', data=df, palette='Set2')
plt.title("DEROG Distribution by Loan Default (BAD)")
plt.xlabel("Loan Default (BAD: 0 = No, 1 = Yes)")
plt.ylabel("DEROG")
plt.show()

### Feature Analysis: `DEROG` – Major Derogatory Reports

**Description:**  
The `DEROG` variable captures the number of **major derogatory credit reports** associated with an applicant. These reports typically reflect **severe negative credit events**, such as bankruptcies, foreclosures, or charge-offs. As such, `DEROG` serves as a strong proxy for historical financial delinquency and is expected to be highly predictive of loan default behavior.

**Distribution Characteristics:**  
- The distribution of `DEROG` is **right-skewed**, with most values concentrated at `0`, indicating no major derogatory records.
- A small portion of applicants exhibit high `DEROG` values, making them natural **outliers**.
- These outliers are **informative rather than erroneous** and should be preserved, particularly for tree-based models which are robust to such skewness.

**Correlation with Target:**  
- `DEROG` shows a **moderate positive correlation** with the target variable `BAD` (loan default), suggesting that applicants with more derogatory reports are more likely to default.

**Treatment Strategy:**  
- For models like **Random Forest** and **XGBoost**, `DEROG` is used **without transformation**, as these models can natively handle skewed numeric features.
- For **logistic regression** or other linear models, it may be beneficial to:
  - Apply a **log-transform** (e.g., `log1p(DEROG)`) to reduce skewness, or
  - Use **binning** to convert the continuous variable into categorical risk bands (e.g., `0`, `1`, `2–3`, `>3`).

**Conclusion:**  
The `DEROG` variable is a **business-critical indicator of creditworthiness** and is retained in its original form for tree-based models. It plays a central role in identifying high-risk applicants and is one of the top-ranked features according to SHAP values.

### Detailed Variable Overview and Outlier Analysis

- **LOAN (Loan Amount):**  
    - **Description:** Represents the approved loan amount.  
    - **Outlier Considerations:** The range is wide, with several extreme high values visible in the boxplot. These outliers may represent unusually large loans that can influence mean-based statistics. Analyzing the histogram reveals right-skewness.

- **MORTDUE (Mortgage Due):**  
    - **Description:** The outstanding amount due on an existing mortgage.  
    - **Outlier Considerations:** Missing values are present and extreme values can be spotted in the boxplot. High values indicate potential risk exposures and may require transformation or capping.

- **VALUE (Property Value):**  
    - **Description:** The current market value of the property used as collateral.  
    - **Outlier Considerations:** Exhibits significant variability with some very high values. Outliers in this variable can distort analyses and may need logarithmic transformation.

- **REASON (Loan Request Purpose):**  
    - **Description:** Categorical variable denoting whether the loan is for home improvement (HomeImp) or debt consolidation (DebtCon).  
    - **Outlier Considerations:** Outlier analysis is less applicable here; however, attention is required regarding rare or missing categories.

- **JOB (Applicant’s Job Type):**  
    - **Description:** Categorical description of the applicant’s employment (e.g., Manager, Office, Other).  
    - **Outlier Considerations:** Similar to REASON, outliers in categorical variables are not defined by extreme numeric values, but missing or rarely occurring classes should be treated carefully.

- **YOJ (Years at Job):**  
    - **Description:** Number of years the applicant has been at their current job.  
    - **Outlier Considerations:** The boxplot may reveal cases with exceptionally long or short job tenures which could be due to data entry issues or genuine outlying behavior.

- **DEROG (Number of Derogatory Reports):**  
    - **Description:** Indicates the count of major derogatory events on the credit report.  
    - **Outlier Considerations:** A relatively low count is expected; however, a few applicants with multiple reports appear as outliers in the boxplot, signaling severe past credit problems.

- **DELINQ (Number of Delinquent Credit Lines):**  
    - **Description:** Represents the number of past delinquent credit lines.  
    - **Outlier Considerations:** Outlier analysis using the IQR method shows that while most applicants have 0 or 1 record, some have unusually higher numbers which could significantly affect risk modeling.

- **CLAGE (Age of Oldest Credit Line):**  
    - **Description:** The age (in months) of the oldest active credit line.  
    - **Outlier Considerations:** Outliers with very high CLAGE values indicate long credit histories. Such extremes are visible in the boxplot and may suggest special credit patterns.

- **NINQ (Number of Recent Inquiries):**  
    - **Description:** The total number of recent credit inquiries.  
    - **Outlier Considerations:** The majority cluster at low values; however, several higher values are evident in the histogram, which might impact the distribution’s tail.

- **CLNO (Number of Credit Lines):**  
    - **Description:** Total count of credit lines the applicant currently holds.  
    - **Outlier Considerations:** Outliers here could represent applicants with an unusually high number of credit accounts, which might be a sign of overextension.

- **DEBTINC (Debt-to-Income Ratio):**  
    - **Description:** Ratio summarizing the debt payments relative to income.  
    - **Outlier Considerations:** The distribution is often skewed, with some extreme values in the boxplot. These high ratios are critical risk indicators and may dominate the untransformed data’s variability.


### Cap values in 'DEBTINC' at the calculated lower and upper bounds

In [ ]:
# Calculate lower and upper bounds for 'DEBTINC' using the IQR method
Q1 = df['DEBTINC'].quantile(0.25)
Q3 = df['DEBTINC'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Cap values in 'DEBTINC' at the calculated lower and upper bounds
df['DEBTINC'] = df['DEBTINC'].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))

# Verify that outliers have been treated
num_below = (df['DEBTINC'] < lower_bound).sum()
num_above = (df['DEBTINC'] > upper_bound).sum()
print("Outliers after treatment: Number of values below lower_bound:", num_below)
print("Outliers after treatment: Number of values above upper_bound:", num_above)

### Univariate Analysis Observations

1. **Continuous Monetary Variables:**
    - Variables such as LOAN, MORTDUE, and VALUE exhibit right-skewed distributions, with most observations concentrated at the lower end and a long tail indicating few very high amounts.
    - The spread in these distributions suggests the presence of significant variability in loan sizes and property values.

2. **Tenure and Age Variables:**
    - YOJ (Years at Present Job) and CLAGE (Age of the Oldest Credit Line) show distributions that are closer to normal but still indicate some skewness.
    - These patterns imply that while most applicants have moderate job tenure and credit history, there are outliers with unusually high or low values.

3. **Downgrade and Delinquency Measures:**
    - Count metrics like DEROG (Number of Derogatory Reports) and DELINQ (Number of Delinquent Credit Lines) are highly concentrated at low counts; most applicants have zero or one report.
    - The distribution of NINQ (Number of Recent Credit Inquiries) similarly indicates that a majority of applicants have very few inquiries.

4. **Implications for Modeling:**
    - The right-skewed nature of monetary variables might benefit from transformations (e.g., log transformation) to reduce the influence of extreme values.
    - The concentrated counts in delinquency measures validate their use as strong indicators for distinguishing between low- and high-risk applicants.

### **Bivariate Analysis relationship with target 'BAD'**

In [ ]:

# Bivariate analysis: relationship with target 'BAD'
plt.figure(figsize=(12, 6))
sns.boxplot(x='BAD', y='LOAN', data=df)
plt.title('Loan Amount by Default Status')
plt.show()

# Cross-tab for REASON
pd.crosstab(df['REASON'], df['BAD'], normalize='index') * 100

#  Loan amount comparison between defaulters and non-defaulters
plt.figure(figsize=(8, 4))
sns.boxplot(x='BAD', y='LOAN', data=df)
plt.title("Loan Amount Comparison: Defaulters vs Non-Defaulters")
plt.xlabel("Defaulted (BAD)")
plt.ylabel("Loan Amount")
plt.show()


#  Mortgage amount comparison between defaulters and non-defaulters
plt.figure(figsize=(8, 4))
sns.boxplot(x='BAD', y='MORTDUE', data=df)
plt.title("Mortgage Amount Comparison: Defaulters vs Non-Defaulters")
plt.xlabel("Defaulted (BAD)")
plt.ylabel("Mortgage Amount")
plt.show()


### Observations from the bivariate analysis on loan amount:

1. The boxplot comparing loan amounts against default status (BAD) shows noticeable differences in distribution between defaulters and non-defaulters.
2. **Non-defaulters generally have a higher median loan amount and a wider range**, suggesting that larger loans are more common among borrowers who do not default.
3. Defaulters tend to have comparatively lower loan amounts; however, there is substantial overlap in the distributions, indicating that **loan amount alone does not fully explain default behavior**.


In [ ]:
# Select key features for panel analysis
panel_features = ['DEBTINC', 'CLAGE', 'DELINQ', 'MORTDUE', 'LOAN', 'VALUE', 'BAD']

# Subset DataFrame for pairplot
pairplot_df = df[panel_features].copy()
pairplot_df = pairplot_df.dropna()  # drop NAs for clean plotting

# Plot pairplot with 'BAD' as hue (target)
sns.set(style="ticks", font_scale=1.1)
g = sns.pairplot(
    pairplot_df,
    hue='BAD',
    palette={0: "tab:blue", 1: "tab:orange"},
    diag_kind="kde",
    plot_kws={'alpha': 0.5, 's': 15},
    corner=True
)
g.fig.suptitle("pairplot with 'BAD'", y=1.02, fontsize=18)
plt.show()


### Observations Based on Panel Distributions

- **Distinct Clustering by Default Status:**  
    The pairplot reveals clear separation between defaulters (BAD = 1) and non‑defaulters (BAD = 0) in key features. While some features like DEBTINC and DELINQ show stronger separation, others overlap more, hinting at complex interdependencies.

- **Variability in Financial Measures:**  
    Features such as LOAN, MORTDUE, and VALUE exhibit wide variances across the applicant pool. Non‑defaulters tend to have a broader spread in loan and property values compared to defaulters, suggesting that high loan amounts are generally associated with safer profiles.

- **Credit Behavior Indicators:**  
    Variables like CLAGE (age of the oldest credit line) and DELINQ (number of delinquent credit lines) display noticeable differences between the two groups. Higher values in these features are associated with an increased likelihood of default, reflecting historical credit challenges.

- **Consistent Patterns Across Features:**  
    The panel distribution confirms that while each feature provides individual insight, their combined visualization presents a richer narrative. Certain attributes, when observed in tandem (for example, high debt-to-income ratio along with multiple delinquencies), highlight clusters of high risk.

- **Usefulness for Feature Engineering:**  
    The observed distribution patterns guide targeted feature transformations and selection. For instance, applying logarithmic transformations to skewed monetary variables or carefully treating outliers could improve model robustness.



### **Multivariate Analysis**

In [ ]:
# Correlation heatmap with significance indicators

# Subset numeric columns from df to avoid converting non-numeric data
numeric_df = df.select_dtypes(include=[np.number])
# Compute correlation matrix from the numeric columns
corr = numeric_df.corr()

# Compute p-values matrix using Pearson correlation on numeric data
pvals = pd.DataFrame(np.ones(corr.shape), columns=corr.columns, index=corr.index)
for row in corr.index:
    for col in corr.columns:
        if row == col:
            pvals.loc[row, col] = 0.0  # Perfect correlation
        else:
            _, p = st.pearsonr(numeric_df[row], numeric_df[col])
            pvals.loc[row, col] = p

# Create annotations: append "*" if p < 0.05
annot_corr = corr.copy()
for i in corr.index:
    for j in corr.columns:
        p_value = pvals.loc[i, j]
        star = "*" if p_value < 0.05 else ""
        annot_corr.loc[i, j] = f"{corr.loc[i, j]:.2f}{star}"

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=annot_corr, cmap="coolwarm", fmt="", square=True)
plt.title("Correlation Heatmap with Significance Indicators (p < 0.05)")
plt.show()

### Observations from the Correlation Heatmap

- **Strong Intercorrelations:**  
    The heatmap shows that monetary variables such as **LOAN**, **MORTDUE**, and **VALUE** are highly correlated. This suggests that multicollinearity may be present, which should be considered during feature engineering and model interpretation.

- **Risk Indicator Relationships:**  
    Features related to credit behavior, such as **DELINQ** and **DEROG**, display moderate correlations with the target variable (**BAD**), reinforcing their potential importance as risk indicators.

- **Numerical Variables Spread:**  
    Other numerical features like **YOJ**, **CLAGE**, and **DEBTINC** have lower correlation values with the target but may still contribute to the model in subtle ways. Their relationships should be further explored to understand complex interactions.

- **Implications for Modeling:**  
    The observed correlations highlight the need for careful preprocessing. Techniques like feature selection or regularization might be useful to mitigate the effects of multicollinearity while retaining the predictive power of the key features.

In [ ]:
# 1. Range of values for the loan amount variable "LOAN"
loan_min = df['LOAN'].min()
loan_max = df['LOAN'].max()
print(f"Range of LOAN: Min = {loan_min}, Max = {loan_max}")


In [ ]:

# 3. Number of unique categories in the REASON variable
unique_reasons = df['REASON'].nunique()
print(f"Number of unique categories in REASON: {unique_reasons}")

# 4. Most common category in the JOB variable
most_common_job = df['JOB'].mode()[0]
print(f"Most common category in JOB: {most_common_job}")


# 7. Correlation between property value and loan default rate
correlation_property_bad = df[['VALUE', 'BAD']].corr().iloc[0, 1]
print(f"Correlation between property value and loan default rate: {correlation_property_bad}")

###  Default Rates by Loan Purpose

In [ ]:

# 5. Relationship between REASON and proportion of applicants who defaulted
reason_default_rate = df.groupby('REASON')['BAD'].mean()
print("\nProportion of applicants who defaulted by REASON:")
print(reason_default_rate)



The analysis shows that the default rate for applicants applying for Debt Consolidation (DebtCon) is approximately 18.97%, whereas for Home Improvement (HomeImp) it is about 22.25%. This indicates that loans requested for Home Improvement carry a slightly higher risk of default compared to those for Debt Consolidation.

### Observations

1. **Correlation with Target Variable (`BAD`)**:
   - The target variable `BAD` shows moderate positive correlations with delinquency-related variables:
     - `DELINQ` has the highest positive correlation (0.354) with `BAD`, suggesting that a higher number of delinquent credit lines is associated with a higher risk of default.
     - `DEROG` also shows a positive correlation (0.276) with `BAD`, indicating that major derogatory reports are linked to default risk.

2. **Numerical and Categorical Variables**:
   - The loan amount (`LOAN`) varies widely, ranging from **1,100** to **89,900**, which could imply segmentation in loan sizes.
   - The most common category in the `JOB` variable is **"Other"**, indicating that many applicants might fall into a general or less defined job category.

3. **Default Rates by Loan Purpose (`REASON`)**:
   - The `REASON` variable, which indicates the loan request reason (`DebtCon` and `HomeImp`), has different default rates:
     - Approximately **18.97%** for `DebtCon`.
     - Approximately **22.25%** for `HomeImp`.
   - This insight could help in tailoring risk models based on the purpose of the loan.





### Class Imbalance Analysis

In [ ]:
# Check class distribution of target variable 'BAD'


# Count values
class_counts = df['BAD'].value_counts()
class_percent = df['BAD'].value_counts(normalize=True) * 100

# Display class distribution
print("Class Distribution:")
print(class_counts)
print("\nPercentage Distribution:")
print(class_percent)

# Plot class distribution
plt.figure(figsize=(6,4))
sns.countplot(x='BAD', data=df, palette='viridis')
plt.title('Target Variable Distribution: BAD (Loan Default)')
plt.xlabel('BAD (0 = No Default, 1 = Default)')
plt.ylabel('Count')
plt.show()


### Treating imbalance

The dataset shows a class imbalance, with approximately 80% of the cases labeled as `0` (no default) and only 20% as `1` (default). This imbalance could lead to biased model performance favoring the majority class.

To address this, we apply **SMOTE (Synthetic Minority Oversampling Technique)** to balance the classes in the training data.

## **Model Building - Approach**
- Data preparation
- Partition the data into train and test set
- Build the model
- Fit on the train data
- Tune the model
- Test the model on test set

## **Data Preprocessing**

- Imputation for missing values was performed
- In this section :
1) Label encoding for categorical features.
2) Scaling of numerical features.


In [ ]:

# Drop rows with missing target
df = df[df['BAD'].notna()]

In [ ]:
# Splitting data into features and target
X = df.drop('BAD', axis=1)
y = df['BAD']


# Train-test split

In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### Justification for Test Size Selection

Choosing the right `test_size` is a trade-off, especially in imbalanced classification problems where SMOTE is applied only to the training set:

- A **test_size of 0.3 (30%)** allows for more examples of the minority class (`BAD = 1`) to remain in the test set. This is critical, because although SMOTE balances the training set, the model is evaluated on the real-world, imbalanced distribution.
- Having a sufficient number of minority-class examples in the test set ensures reliable estimates of metrics such as **recall**, **F1-score**, and **ROC-AUC**, which are particularly sensitive to class imbalance.
- A **test_size of 0.2 (20%)**, while allocating more data to training, can make the evaluation unreliable due to too few positive (default) samples in the test set.

**Recommendation:** With ~6000 rows and only ~20% default cases, using `test_size=0.3` ensures a meaningful and balanced evaluation phase, without overly compromising training size.


## Feature Preprocessing with ColumnTransformer

To prepare the features for modeling, a column-wise preprocessing pipeline is constructed using `ColumnTransformer`.  
- **Numerical features** are standardized to zero mean and unit variance using `StandardScaler`, which often improves convergence and model interpretability.
- **Categorical features** are one-hot encoded (with the first category dropped to avoid multicollinearity), allowing models to treat categorical values as separate, independent features.

This modular pipeline ensures that all subsequent modeling steps use the same, correctly preprocessed feature space.


In [ ]:

cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
])
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.fit_transform(X_test)



## **Base Modeling: Logistic Regression, Random Forest, and XGBoost**

We train and evaluate three classification models:
- Logistic Regression (interpretable baseline).
- Random Forest (ensemble of decision trees).
- XGBoost (gradient boosting technique).


In [ ]:

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_proc, y_train)
lr_preds = lr.predict(X_test_proc)

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_proc, y_train)
rf_preds = rf.predict(X_test_proc)

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_proc, y_train)
xgb_preds = xgb.predict(X_test_proc)

# Evaluation
print("Logistic Regression:")
print(classification_report(y_test, lr_preds))
print("ROC-AUC:", roc_auc_score(y_test, lr.predict_proba(X_test_proc)[:, 1]))

print("Random Forest:")
print(classification_report(y_test, rf_preds))
print("ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test_proc)[:, 1]))

print("XGBoost:")
print("ROC-AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test_proc)[:, 1]))
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, ConfusionMatrixDisplay

# Logistic Regression
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr.predict_proba(X_test_proc)[:, 1])
lr_auc = auc(lr_fpr, lr_tpr)

# Random Forest
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf.predict_proba(X_test_proc)[:, 1])
rf_auc = auc(rf_fpr, rf_tpr)

# XGBoost
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, xgb.predict_proba(X_test_proc)[:, 1])
xgb_auc = auc(xgb_fpr, xgb_tpr)

# Plot ROC Curves
plt.figure(figsize=(10, 6))
plt.plot(lr_fpr, lr_tpr, label=f"Logistic Regression (AUC = {lr_auc:.2f})")
plt.plot(rf_fpr, rf_tpr, label=f"Random Forest (AUC = {rf_auc:.2f})")
plt.plot(xgb_fpr, xgb_tpr, label=f"XGBoost (AUC = {xgb_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid()
plt.show()

# Confusion Matrices
models = {'Logistic Regression': lr, 'Random Forest': rf, 'XGBoost': xgb}
for name, model in models.items():
    print(f"Confusion Matrix for {name}:")
    ConfusionMatrixDisplay.from_estimator(model, X_test_proc, y_test, cmap='Blues')
    plt.title(f"Confusion Matrix: {name}")
    plt.show()

### **Evaluation of Model Performance with Visualizations**

- **Logistic Regression:**
    - **Accuracy**: 0.84
    - **Macro-average F1-score**: 0.68
    - **ROC-AUC**: 0.7624
    - **Observations from Visualizations**:
        - The ROC curve for Logistic Regression shows a moderate ability to distinguish between defaulters and non-defaulters, with an AUC of 0.76.
        - The confusion matrix indicates that while the model performs well for the majority class (non-defaulters), it struggles to correctly classify defaulters (class 1), leading to lower recall for this class.
    - **Summary**: Logistic Regression provides a good baseline but lacks the ability to effectively identify defaulters, as seen in the ROC curve and confusion matrix.

- **Random Forest:**
    - **Accuracy**: 0.91
    - **F1-score for default class (1.0)**: 0.75
    - **ROC-AUC**: 0.9592
    - **Observations from Visualizations**:
        - The ROC curve for Random Forest is steep and close to the top-left corner, indicating excellent discrimination between defaulters and non-defaulters, with an AUC of 0.96.
        - The confusion matrix shows a significant improvement in correctly classifying defaulters compared to Logistic Regression, with fewer false negatives.
    - **Summary**: Random Forest demonstrates strong performance, effectively balancing precision and recall for both classes, as reflected in the ROC curve and confusion matrix.

- **XGBoost:**
    - **ROC-AUC**: 0.9528
    - **Observations from Visualizations**:
        - The ROC curve for XGBoost is very similar to that of Random Forest, with an AUC of 0.95, indicating competitive performance.
        - The confusion matrix highlights that XGBoost also performs well in identifying defaulters, with a slightly higher number of false negatives compared to Random Forest.
    - **Summary**: XGBoost is a robust model with performance comparable to Random Forest. Its slightly lower recall for defaulters compared to Random Forest is reflected in the confusion matrix.

---

### **Overall Summary**
- **ROC Curves**: The ROC curves clearly show that ensemble methods (Random Forest and XGBoost) outperform Logistic Regression in terms of discrimination power, with Random Forest achieving the highest AUC.
- **Confusion Matrices**: Both Random Forest and XGBoost significantly reduce false negatives compared to Logistic Regression, making them more effective for identifying defaulters.
- Random Forest is the best-performing model based on ROC-AUC, confusion matrix insights, and overall classification metrics. It is well-suited for this loan default prediction task, offering a balance between accuracy and interpretability.

## Model Training and Evaluation Framework

### Modular Experiment Management

All experiments are managed using the custom `ExperimentManager` class, which provides a modular and reproducible framework for machine learning model development. This class supports:

- Model registration with consistent preprocessing and optional SMOTE oversampling for class imbalance.
- Automated hyperparameter tuning using cross-validated grid search.
- Unified model evaluation and reporting of key classification metrics.
- Advanced statistical comparison between models.

### Handling Class Imbalance

Each model is configured with `class_weight='balanced'`, ensuring that the learning algorithm compensates for class imbalance during training. This parameter automatically adjusts the weights inversely proportional to class frequencies in the data, making misclassifications of the minority class more costly for the model. As a result, the model is less biased towards the majority class and is better able to recognize patterns from the minority (target) class, which is critical for imbalanced classification tasks.

### Model Pipelines

The following models are included in the evaluation, all with `class_weight='balanced'`:

- **XGBoost**
- **Random Forest**
- **Logistic Regression**

Each model is integrated into a pipeline with:
- Common data preprocessing steps,
- Optional SMOTE (Synthetic Minority Over-sampling Technique) for further class balancing,
- Grid search over relevant hyperparameters.

### Experimental Workflow

1. **Model Initialization**  
   Models are initialized with `class_weight='balanced'`, ensuring fair treatment of both classes during training.

2. **Model Training and Hyperparameter Tuning**  
   Each model undergoes hyperparameter tuning using stratified cross-validation and the macro F1-score as the optimization metric.

3. **Evaluation**  
   The best estimators are evaluated on a separate test set. Performance is measured using accuracy, precision, recall, F1-score, and ROC AUC.

4. **Cross-Validation and Statistical Comparison**  
   Cross-validated results are visualized and statistically compared (Wilcoxon, t-test, Kruskal-Wallis, McNemar, Cochran’s Q, ANOVA) to assess the significance of differences between models.

### Summary

The combined use of `class_weight='balanced'` and SMOTE ensures robust handling of class imbalance both at the data level (SMOTE) and the algorithmic level (`class_weight='balanced'`). This setup provides a fair evaluation of each model’s true discriminative power, especially for the minority class, which is of primary interest in this project.


In [ ]:
class ExperimentManager:
    """
    Modular experiment manager for ML model pipelines.
    Handles model registration, hyperparameter tuning, evaluation,
    cross-validation, and statistical comparison.
    """
    def __init__(self, preprocessor, cv_splits=5, random_state=42):
        """
        Initialize manager with a preprocessor, cross-validation setup, and random state.
        """
        self.preprocessor = preprocessor
        self.cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
        self.random_state = random_state
        self.models = {}
        self.best_estimators_ = {}
        self.best_params_ = {}
        self.best_scores_ = {}

    def add_model(self, name, estimator, param_grid=None, use_smote=False):
        """
        Add a new model pipeline, with optional SMOTE and hyperparameter grid.

        Parameters:
            name (str): Name of the model.
            estimator: Estimator/classifier instance.
            param_grid (dict): Grid for hyperparameter tuning.
            use_smote (bool): Whether to apply SMOTE.
        """
        steps = [('preprocessor', self.preprocessor)]
        if use_smote:
            steps.append(('smote', SMOTE(random_state=self.random_state)))
        steps.append(('clf', estimator))
        pipeline = ImbPipeline(steps) if use_smote else SklearnPipeline(steps)
        self.models[name] = {'pipeline': pipeline, 'param_grid': param_grid}

    def tune(self, X, y, scoring='roc_auc', use_cv=True):
        """
        Tune models using GridSearchCV or train directly if no param_grid.
        Stores the best estimators for downstream evaluation.

        Parameters:
            X: Training features.
            y: Training labels.
            scoring (str): Scoring metric for GridSearchCV.
            use_cv (bool): Whether to use cross-validation for hyperparameter tuning.
        """
        # Clear previous results if any
        self.best_estimators_ = {}
        self.best_params_ = {}
        self.best_scores_ = {}
        for name, cfg in self.models.items():
            pipe = cfg['pipeline']
            if use_cv and cfg['param_grid']:
                gs = GridSearchCV(pipe, cfg['param_grid'], cv=self.cv,
                                  scoring=scoring, n_jobs=-1)
                gs.fit(X, y)
                self.best_estimators_[name] = gs.best_estimator_
                self.best_params_[name] = gs.best_params_
                self.best_scores_[name] = gs.best_score_
                print(f"{name} best {scoring}: {gs.best_score_:.4f}")
            else:
                pipe.fit(X, y)
                self.best_estimators_[name] = pipe
                self.best_params_[name] = None
                self.best_scores_[name] = None
                print(f"{name} trained without CV")
                pipe.fit(X, y)
                self.best_estimators_[name] = pipe
                print(f"{name} trained without CV")

    def evaluate(self, X_train, X_test_proc, y_train, y_test):
        """
        Evaluate best estimators on holdout data.
        Returns DataFrame with accuracy, precision, recall, f1, and roc_auc.
        """
        results = []
        for name, est in self.best_estimators_.items():
            est.fit(X_train, y_train)
            y_pred = est.predict(X_test_proc)
            y_prob = est.predict_proba(X_test_proc)[:,1] if hasattr(est, 'predict_proba') else None
            results.append({
                'model': name,
                'accuracy': accuracy_score(y_test, y_pred),
                'precision': precision_score(y_test, y_pred),
                'recall': recall_score(y_test, y_pred),
                'f1': f1_score(y_test, y_pred),
                'roc_auc': roc_auc_score(y_test, y_prob) if y_prob is not None else None
            })
        return pd.DataFrame(results).set_index('model')

    def cross_val_scores(self, X, y, scoring):
        """
        Compute cross-validation scores for each model.
        Returns dict: model name -> scores for each CV fold.
        """
        scores = {}
        for name, cfg in self.models.items():
            scorer = make_scorer(scoring) if callable(scoring) else scoring
            cv_scores = cross_val_score(
                cfg['pipeline'], X, y, cv=self.cv, scoring=scorer, n_jobs=-1
            )
            scores[name] = cv_scores
        return scores

# --- Model registration ---
mgr = ExperimentManager(preprocessor)

""" Register Xgboost, Random Forest, and Logistic Regression models
    with respective hyperparameter grids and SMOTE balancing. """
mgr.add_model(
    'XGBoost',
    XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss',
        tree_method='hist',
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()  # Handle imbalance
    ),
    param_grid={
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [3, 6, 10],
        'clf__learning_rate': [0.01, 0.1, 0.3],
        'clf__subsample': [0.8, 1.0],
        'clf__colsample_bytree': [0.8, 1.0],
        'clf__reg_alpha': [0, 0.5],
        'clf__reg_lambda': [1, 2]
    },
    use_smote=True
)
mgr.add_model(
    'RandomForest',
 RandomForestClassifier(random_state=42, class_weight='balanced'),
    param_grid={'clf__n_estimators': [100, 200],
                'clf__max_depth': [10, 20],
                'clf__min_samples_split': [2, 5],
                'clf__min_samples_leaf': [1, 2],
                'clf__bootstrap': [True, False],
                'clf__criterion': ['gini', 'entropy']},
    use_smote=True
)
mgr.add_model(
    'LogisticRegression',
    LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
    param_grid={
    'clf__penalty': ['l2'],
    'clf__C': [0.1, 1, 10],
    'clf__solver': ['liblinear', 'lbfgs'],
    'clf__max_iter': [500]},
    use_smote=True
)

""" Perform hyperparameter tuning using cross-validation (GridSearchCV). """
mgr.tune(X_train, y_train, scoring='f1_macro', use_cv=True)


print("Best hyperparameters and CV scores for each model:\n")
for name in mgr.models:
    if mgr.best_params_[name] is not None:
        print(f"{name}:")
        print("  Best hyperparameters:", mgr.best_params_[name])
        print(f"  Best CV score: {mgr.best_scores_[name]:.4f}\n")
    else:
        print(f"{name}: No hyperparameter tuning was performed.\n")

""" Evaluate final models on test set and report main classification metrics. """
results_df = mgr.evaluate(X_train, X_test, y_train, y_test)
print("\nPerformance after tuning - GridSearch:\n", results_df)

""" Cross-validation: compute F1 (macro) scores for each model across all folds. """
cv_scores = mgr.cross_val_scores(
    X_train, y_train,
    scoring=lambda yt, yp: f1_score(yt, yp, average='macro')
)

""" Visualize CV score distributions for each model. """
plt.figure(); sns.boxplot(data=list(cv_scores.values()));
plt.xticks(range(len(cv_scores)), list(cv_scores.keys()));
plt.title('F1 (macro) across CV folds'); plt.show()

# --- Statistical comparison ---
""" Compare models statistically: Wilcoxon, t-test, Kruskal-Wallis. """
### Define helper function for p-value formatting
def p_fmt(p):
    s = f"{p:.3f}"
    return "<0.0001" if s == "0.000" else s

""" Compare models statistically: Wilcoxon, t-test, Kruskal-Wallis. """
names = list(cv_scores.keys())
stat_w, p_w = st.wilcoxon(cv_scores[names[0]], cv_scores[names[1]])
stat_t, p_t = st.ttest_ind(cv_scores[names[0]], cv_scores[names[1]])
print(f"\nWilcoxon {names[0]} vs {names[1]}: stat={stat_w:.3f}, p={p_fmt(p_w)}")
print(f"T-test {names[0]} vs {names[1]}: stat={stat_t:.3f}, p={p_fmt(p_t)}")
stat_k, p_k = st.kruskal(*cv_scores.values())
print(f"Kruskal-Wallis: stat={stat_k:.3f}, p={p_fmt(p_k)}")

""" Predict on test set and compute McNemar and Cochran's Q for predictions. """
preds = {}
for name, est in mgr.best_estimators_.items():
    est.fit(X_train, y_train)
    preds[name] = est.predict(X_test)

tbl = mcnemar_table(y_test, preds[names[0]], preds[names[1]])
chi2_m, p_m = mcnemar(tbl)
print(f"\nMcNemar {names[0]} vs {names[1]}:\n", tbl)
print(f"chi2={chi2_m:.3f}, p={p_m:.3f}")

cq_stat, cq = cochrans_q(y_test.to_numpy(), *[preds[n] for n in names])
print(f"\nCochran's Q: {cq:.3f}")

""" ANOVA-based comparison for CV scores (Welch ANOVA + Games-Howell post-hoc). """
t_scores = np.hstack([cv_scores[n] for n in names])
models_col = sum([[n]*len(cv_scores[n]) for n in names], [])
df_scores = pd.DataFrame({'cv_score': t_scores, 'model': models_col})
anova = pg.welch_anova(data=df_scores, dv='cv_score', between='model')
posthoc = pg.pairwise_gameshowell(data=df_scores, dv='cv_score', between='model')
print("\nWelch ANOVA:\n", anova)
print("\nPost-hoc Games-Howell:\n", posthoc)

# Save the best model
best_model_name = max(mgr.best_scores_, key=mgr.best_scores_.get)
best_model = mgr.best_estimators_[best_model_name]
joblib.dump(best_model, 'best_model.pkl')



## Statistical Comparison of Model Performance

To ensure the observed differences between models are meaningful and not due to random variation, several statistical tests were performed. Here is an interpretation of the results:

---

### 1. **Pairwise Tests Between XGBoost and Random Forest**

- **Wilcoxon Test**:  
  - Statistic = 3.000, p = 0.312  
  - *Interpretation*: The Wilcoxon signed-rank test checks for significant differences in the paired cross-validation scores between XGBoost and Random Forest.  
  - **Conclusion:** p > 0.05, so we do **not** find a statistically significant difference in median performance between these models.

- **T-Test**:  
  - Statistic = 0.487, p = 0.639  
  - *Interpretation*: The independent t-test compares the mean scores across cross-validation folds.
  - **Conclusion:** Again, p > 0.05 means no statistically significant difference in means.

---

### 2. **Kruskal-Wallis Test**

- **Statistic = 9.620, p = 0.008**  
- *Interpretation*: This non-parametric test checks whether at least one model out of the group has a distribution of scores different from the others.
- **Conclusion:** p < 0.05 indicates a statistically significant difference exists **somewhere** among the models, prompting post-hoc pairwise comparisons.

---

### 3. **McNemar’s Test (XGBoost vs Random Forest)**

- **Contingency Table:**  
  ```
  [[1584   66]
   [  30  108]]
  ```
- **chi2 = 12.76, p = 0**
- *Interpretation*: McNemar’s test assesses the differences in classification errors between two models on the same test set.
- **Conclusion:** p < 0.05 shows there is a significant difference in the pattern of misclassifications.  
  This suggests that while overall metrics (like mean F1) may be similar, the models make errors on different samples.

---

### 4. **Cochran’s Q Test**

- **Statistic = 0**
- *Interpretation*: This tests whether there are differences in performance among multiple classifiers across the same dataset.
- **Conclusion:** Here, the value indicates no significant difference detected—possibly due to the specific configuration or high agreement among predictions.

---

### 5. **Welch ANOVA & Post-hoc Games-Howell**

- **Welch ANOVA**:  
  - F = 105.11, p = 0.000002  
  - *Interpretation*: Welch’s ANOVA is robust to unequal variances and tests whether mean performance differs among models.
  - **Conclusion:** Very low p-value confirms a statistically significant difference between at least some model means.

- **Post-hoc Games-Howell**:  
  - Compares all pairs, accounting for variance differences.  
  - *Key results*:  
    - Logistic Regression is significantly outperformed by both Random Forest and XGBoost (very low p-values).
    - XGBoost and Random Forest: mean difference is small and **not statistically significant** (p = 0.88).
  - **Conclusion:** The main difference is between the ensemble models and Logistic Regression; between XGBoost and Random Forest, differences are minimal.

---

## **Summary of Statistical Insights**

- Both XGBoost and Random Forest are **significantly better** than Logistic Regression.
- **No statistically significant difference** in overall cross-validated F1 scores between XGBoost and Random Forest—either can be justified for deployment based on these metrics.
- **However, McNemar’s test** reveals that the pattern of errors made by XGBoost and Random Forest is **statistically different**—they do not make mistakes on the same samples. This could be leveraged for model ensembling or for selecting a model based on specific business needs.
- **Welch ANOVA and Games-Howell** reinforce that the biggest leap in performance is from linear to ensemble models, not between the ensembles themselves.

---

**In conclusion:**  
All advanced statistical tests confirm that ensemble methods (XGBoost, Random Forest) are vastly superior to Logistic Regression for this problem, but differences between the two ensembles are small in practical terms. Model choice between XGBoost and Random Forest can therefore be based on interpretability, speed, or integration needs rather than pure performance.

---



# Why Macro F1 and ROC AUC Are Preferred Here
Macro F1 considers both classes equally, making it the most robust metric for imbalanced datasets—where both missing defaulters and misclassifying good customers matter.

ROC AUC evaluates model discrimination across all thresholds, providing an overall sense of model skill regardless of the decision cutoff.

In [ ]:

diff = np.array(cv_scores["XGBoost"]) - np.array(cv_scores["RandomForest"])
plt.figure(figsize=(8,4))
plt.plot(diff, marker='o', linestyle='-', color='teal')
plt.axhline(0, color='red', linestyle='--')
plt.title("Fold-wise Macro F1 Differences (XGBoost - Random Forest)")
plt.xlabel("CV Fold")
plt.ylabel("F1 Difference")
plt.grid(True)
plt.show()


### Observations Based on Fold-wise Macro F1 Scores ###

In [ ]:


# McNemar contingency table (from your output)
tbl = np.array([[1584, 66],
                [30, 108]])

plt.figure(figsize=(5,4))
sns.heatmap(tbl, annot=True, fmt='d', cmap='Blues',
            xticklabels=["RF Correct/XGB Wrong", "Both Correct"],
            yticklabels=["Both Wrong", "RF Wrong/XGB Correct"])
plt.title("McNemar's Test Contingency Table\n(XGBoost vs Random Forest)")
plt.ylabel("True Outcome")
plt.xlabel("Prediction Outcome")
plt.show()


### Observations Based on the McNemar Contingency Analysis

- The contingency table shows a high level of agreement between the two models, with a large number of cases where both models either correctly or incorrectly classified the same samples.
- There were 28 instances where one model (e.g., XGBoost) was correct while the other (e.g., Random Forest) misclassified, compared to 19 instances in the opposite direction.
- The relatively small number of discordant pairs, in comparison to the total number of cases, indicates that the error patterns of the two models are quite similar.
- The McNemar test yielded a p-value (≈0.24) that is above the common significance threshold (0.05), suggesting that the differences in their misclassification patterns are not statistically significant.
- Overall, the results imply that both models make largely consistent predictions on the test set, which could be advantageous when considering model ensembling or choosing one model based on other factors such as interpretability or computational efficiency.

In [ ]:
posthoc_labels = [
    "LogReg vs RandomForest",
    "LogReg vs XGBoost",
    "RandomForest vs XGBoost"
]
posthoc_diffs = [-0.176, -0.181, -0.005]
posthoc_errors = [0.013, 0.013, 0.011]

plt.figure(figsize=(8,4))
plt.barh(posthoc_labels, posthoc_diffs, xerr=posthoc_errors, color='skyblue')
plt.axvline(0, color='red', linestyle='--')
plt.title("Games-Howell Pairwise Mean Macro F1 Differences")
plt.xlabel("Mean F1 Difference (A - B)")
plt.tight_layout()
plt.show()

### Observations Based on the Games-Howell Pairwise Mean Macro F1 Differences

- **Logistic Regression vs. Ensemble Models:**  
    Logistic Regression shows a significant drop in macro F1 performance compared to both Random Forest and XGBoost. The mean differences of approximately –0.174 and –0.178 (LogReg vs. Random Forest and LogReg vs. XGBoost, respectively) with extremely low p-values indicate that Logistic Regression consistently underperforms in capturing balanced performance between defaulters and non‑defaulters.

- **Ensemble Methods Comparison:**  
    The mean difference between Random Forest and XGBoost is very small (around –0.00367) with a very high p-value (approximately 0.90), suggesting no statistically significant difference between these two models. Their macro F1 scores are essentially equivalent.

- **Overall Implication:**  
    The post-hoc Games-Howell analysis reinforces that ensemble methods (Random Forest and XGBoost) are superior in terms of macro F1 score compared to the linear approach of Logistic Regression, while there’s no meaningful performance gap between the two ensemble models.

# In the context of loan default prediction, the most critical performance measures are:

**Macro F1 Score**– to ensure balanced performance across both borrower types,

**Recall** – to minimize undetected defaulters, which are most costly to the bank,

**ROC AUC** – to evaluate overall discriminative power of the model.

Metrics like accuracy and precision are included for completeness but are not sufficient for reliable model selection in imbalanced, risk-sensitive tasks.

# Model Comparison
---

## Performance Metrics Overview

| Model               | Accuracy  | Precision  | Recall     | F1 Score   | ROC AUC   |
|---------------------|-----------|------------|------------|------------|-----------|
| **XGBoost**         | 0.928     | 0.821      | 0.821      | 0.821      | 0.961     |
| **Random Forest**   | 0.908     | 0.777      | 0.754      | 0.765      | 0.952     |
| **Logistic Regression** | 0.766 | 0.439      | 0.619      | 0.514      | 0.781     |

- **Best f1_macro (CV):**
    - XGBoost: **0.8776**
    - RandomForest: **0.8610**
    - LogisticRegression: **0.6850**

- **Best Hyperparameters:**  
    - *XGBoost*: colsample_bytree=1.0, learning_rate=0.3, max_depth=6, n_estimators=200, reg_alpha=0.5, reg_lambda=1, subsample=0.8  
    - *RandomForest*: bootstrap=False, criterion=gini, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=200  
    - *LogisticRegression*: C=1, max_iter=500, penalty=l2, solver=liblinear

---

## Observations and Interpretation

### 1. **Model Discrimination and Generalization**
- **XGBoost** outperforms the other models across all metrics. With an F1 score of 0.82 and ROC AUC of 0.96, it demonstrates both high accuracy and robust class separation, which is essential for predicting defaults (imbalanced classification). This is reinforced by the highest CV f1_macro.
- **Random Forest** is a strong baseline, also showing excellent ROC AUC (0.95), but lags slightly behind XGBoost in F1 and recall, suggesting marginally weaker detection of defaulters.
- **Logistic Regression** underperforms significantly, with an F1 score barely above 0.5 and much lower precision. This suggests that the linear model is insufficiently expressive for the complexity of the relationships in the data.

### 2. **Business Impact**
- **False Negatives** (predicting non-default when a default occurs) are especially costly for the bank. Both XGBoost and Random Forest offer a much better balance between precision and recall compared to Logistic Regression, reducing this business risk.
- **Interpretability** is crucial per project requirements. While XGBoost and Random Forest are less transparent than Logistic Regression, feature importance techniques (e.g., SHAP, permutation importance) can provide necessary justification for decisions, as required by banking regulation.

### 3. **Hyperparameter Tuning**
- Substantial gains are realized via hyperparameter tuning, especially with XGBoost (notably, high values for n_estimators and a moderate learning rate).
- Both ensemble models favor deeper trees and more estimators, suggesting complex interactions in the data.

### 4. **Recommendations for Final Solution**
- **XGBoost** is recommended as the primary model for deployment due to superior predictive performance. Interpretability tools must be integrated to meet regulatory requirements.
- Further marginal improvements may be possible via ensemble stacking or additional feature engineering.
- Logistic Regression, though interpretable, is not suitable as a primary model given current results. However, its coefficients can be used for supplementary insights into linear feature relationships.

---

## Conclusion and Next Steps

- **XGBoost** provides the best trade-off between predictive power and business requirements. Its adoption would help the bank minimize losses due to default, while feature attribution methods can maintain model transparency.
- For production, monitoring of model drift, regular retraining, and integration with interpretability frameworks are strongly recommended.
- Continued evaluation using business-focused metrics (e.g., cost of misclassification, profit curves) may be advisable to align model performance with bank objectives.

---

## Evaluation of the Best Model: ROC-AUC and Confusion Matrix

Below, we evaluate the best-performing model (as selected by cross-validated score) on the holdout test set. We report the ROC-AUC value and present both the ROC curve and the confusion matrix to illustrate the model's discriminatory power and error distribution.


In [ ]:
# Predict probabilities and classes
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate ROC-AUC
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC of the best model ({best_model_name}): {roc_auc:.4f}")

# Plot ROC Curve and Confusion Matrix side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
axes[0].plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
axes[0].plot([0, 1], [0, 1], 'k--', label="Random Guess")
axes[0].set_xlabel("False Positive Rate")
axes[0].set_ylabel("True Positive Rate")
axes[0].set_title(f"ROC Curve - {best_model_name}")
axes[0].legend(loc="lower right")
axes[0].grid(True)

# Confusion Matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=axes[1], cmap="Blues", colorbar=False)
axes[1].set_title(f"Confusion Matrix - {best_model_name}")

plt.tight_layout()
plt.show()

# Classification report for completeness
print(f"\nClassification report for the best model ({best_model_name}):\n")
print(classification_report(y_test, y_pred))


## Saving the Best Model and Interpreting Predictions with SHAP

After hyperparameter tuning and evaluation, the best-performing model is identified and saved for future deployment.  
To ensure regulatory compliance and business transparency, SHAP (SHapley Additive exPlanations) analysis is conducted to interpret the feature impact on the model's predictions.

Model Saving:
Saves the entire pipeline (preprocessing + model). This guarantees you can load and use the model on raw data in production.

SHAP:
Uses the trained classifier on the correctly preprocessed data, ensuring explanations match the actual prediction process.

Feature Names:
Handles both numeric and one-hot-encoded categorical features, improving interpretability of the summary plot.

Visualization:
The summary plot gives a global view of which features drive predictions (for compliance and business explanations).

In [ ]:

# 1. Find the best model according to validation scores
best_model_name = max(mgr.best_scores_, key=mgr.best_scores_.get)
best_pipeline = mgr.best_estimators_[best_model_name]
print(f"Best model: {best_model_name}")

# 2. Save the full pipeline (preprocessing + model) for deployment
joblib.dump(best_pipeline, 'best_model.pkl')

# To load later: loaded_pipeline = joblib.load('best_model.pkl')

# 3. Prepare data for SHAP analysis
# Extract the classifier and the fitted preprocessor
clf = best_pipeline.named_steps['clf']
preprocessor = best_pipeline.named_steps['preprocessor']

# Transform train and test data to match model input
X_train_shap = preprocessor.transform(X_train)
X_test_shap = preprocessor.transform(X_test)

# 4. Build the SHAP explainer
# For tree-based models, SHAP will use TreeExplainer automatically
explainer = shap.Explainer(clf, X_train_shap)

# 5. Compute SHAP values for the test set
shap_values = explainer(X_test_shap)

# 6. Visualize feature importance (SHAP summary plot)
# Use original feature names for interpretability
try:
    num_feature_names = preprocessor.named_transformers_['num'].get_feature_names_out(num_cols).tolist()
    cat_feature_names = []
    if cat_cols:  # only try to extract names if there are categorical columns
        cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols).tolist()
    feature_names = num_feature_names + cat_feature_names
except Exception:
    # Fallback: Just use column names if get_feature_names_out is unavailable
    feature_names = num_cols + cat_cols

shap.summary_plot(shap_values, X_test_shap, feature_names=feature_names)




### SHAP Summary Plot – Key Observations

- **DEBTINC (Debt-to-Income Ratio) is the most influential feature** in the model. High debt-to-income values strongly increase the predicted risk of loan default, as indicated by the concentration of red points with large positive SHAP values.

- **CLAGE (Age of Oldest Credit Line) also plays a significant role**. The model indicates that applicants with a longer credit history (higher CLAGE) have a higher risk of default in this dataset, which may be a reflection of specific risk profiles in the population.

- **DELINQ (Number of Delinquencies) has a strong positive effect** on default risk. Applicants with more previous delinquencies are much more likely to default, as shown by the rightward shift of red dots.

- **Features such as YOJ (Years at Current Job), CLNO (Number of Credit Lines), LOAN, VALUE, and MORTDUE** contribute to predictions but have a smaller overall impact compared to the top features. Their effects are more nuanced and less pronounced.

- **DEROG (Major Derogatory Reports) and NINQ (Number of Recent Inquiries)**, while included in the model, show relatively limited influence on the model’s output compared to the other features.

- **General Pattern:**  
  - Features where high values (red) appear on the right increase the risk of default when their value is high.
  - Features where low values (blue) appear on the left decrease the risk of default.

- **Model Interpretability:**  
  - The SHAP plot provides transparency, allowing stakeholders to understand which variables drive the model's decisions.
  - These insights support regulatory compliance and help prioritize risk factors for further business investigation.

**Conclusion:**  
The SHAP analysis confirms that the model's predictions are most strongly driven by the applicant’s debt-to-income ratio, age of credit history, and prior delinquencies. These features should be closely monitored in both model development and business decision-making.


In [ ]:
print(feature_names)
print(len(feature_names), X_test_shap.shape[1])

In [ ]:
X_test_df  = pd.DataFrame(X_test_shap,  columns=feature_names)
shap_values = explainer(X_test_df)
shap.plots.bar(shap_values, max_display=8)

### Observations from the SHAP Bar Plot

- **DEBTINC Dominance:**  
    The Debt-to-Income Ratio (DEBTINC) stands out with the highest average absolute SHAP value, indicating it is the most influential feature driving the model’s predictions. Higher values of DEBTINC are strongly associated with an increased risk of loan default.

- **Importance of DELINQ and CLAGE:**  
    The number of Delinquent Credit Lines (DELINQ) and the age of the oldest Credit Line (CLAGE) also emerge as key contributors. Their relatively high SHAP values suggest that more past delinquencies and longer credit histories significantly impact the prediction towards default.

- **Moderate Contribution of Other Features:**  
    Other features (such as MORTDUE and DEROG) contribute to the model’s output, but their impact is notably lower compared to DEBTINC, DELINQ, and CLAGE.

- **Overall Insight:**  
    The SHAP analysis confirms that the model largely bases its decisions on financial stability indicators, particularly the borrower’s debt load and credit behavior. This insight is critical for understanding the risk factors driving loan default predictions.

# Fit XGBoost classifier with best hyperparameters to most important features

In [ ]:

# 1. Define selected SHAP features
selected_features = ['DEBTINC', 'CLAGE', 'DELINQ']

# 2. Prepare data with only these features from the original DataFrame
X_train_sel = X_train[selected_features]
X_test_proc_sel = X_test[selected_features]

# 3. Identify column types for proper preprocessing from the DataFrame
cat_cols = X_train_sel.select_dtypes(include='object').columns.tolist()
num_cols = X_train_sel.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 4. Create preprocessing transformer for selected features
preprocessor_shap = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
])

# 5. Set class weight for imbalance (as in previous pipeline)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# 6. Create the full pipeline: preprocessing + SMOTE + classifier
pipeline_shap = ImbPipeline([
    ('preprocessor', preprocessor_shap),
    ('smote', SMOTE(random_state=42)),
    ('clf', XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight,
        tree_method='hist'
    ))
])
# 7. Fit the pipeline and evaluate on test data using the DataFrame with selected features
pipeline_shap.fit(X_train_sel, y_train)
y_pred_shap = pipeline_shap.predict(X_test_proc_sel)
y_proba_shap = pipeline_shap.predict_proba(X_test_proc_sel)[:, 1]

print(selected_features)
print(classification_report(y_test, y_pred_shap))
print(f"Accuracy:  {accuracy_score(y_test, y_pred_shap):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_shap):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_shap):.4f}")
print(f"F1-score:  {f1_score(y_test, y_pred_shap):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_shap):.4f}")


In [ ]:
mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
feature_ranking = np.argsort(mean_abs_shap)[::-1]

# Teraz feature_names[feature_ranking[:N]] to NAJWAŻNIEJSZE cechy z punktu widzenia modelu
for N in range(3, 6):
    top_features = [feature_names[i] for i in feature_ranking[:N]]
    print(f"Top {N} SHAP features (po transformacji): {top_features}")


In [ ]:

# 1. Define selected SHAP features
selected_features = ['DEBTINC', 'DELINQ', 'CLAGE', 'MORTDUE']

# 2. Prepare data with only these features from the original DataFrame
X_train_sel = X_train[selected_features]
X_test_proc_sel = X_test[selected_features]

# 3. Identify column types for proper preprocessing from the DataFrame
cat_cols = X_train_sel.select_dtypes(include='object').columns.tolist()
num_cols = X_train_sel.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 4. Create preprocessing transformer for selected features
preprocessor_shap = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
])

# 5. Set class weight for imbalance (as in previous pipeline)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# 6. Create the full pipeline: preprocessing + SMOTE + classifier
pipeline_shap = ImbPipeline([
    ('preprocessor', preprocessor_shap),
    ('smote', SMOTE(random_state=42)),
    ('clf', XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight,
        tree_method='hist'
    ))
])
# 7. Fit the pipeline and evaluate on test data using the DataFrame with selected features
pipeline_shap.fit(X_train_sel, y_train)
y_pred_shap = pipeline_shap.predict(X_test_proc_sel)
y_proba_shap = pipeline_shap.predict_proba(X_test_proc_sel)[:, 1]

print(selected_features)
print(classification_report(y_test, y_pred_shap))
print(f"Accuracy:  {accuracy_score(y_test, y_pred_shap):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_shap):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_shap):.4f}")
print(f"F1-score:  {f1_score(y_test, y_pred_shap):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_shap):.4f}")

In [ ]:

# 1. Define selected SHAP features
selected_features = ['DEBTINC', 'DELINQ', 'CLAGE', 'MORTDUE', 'DEROG']

# 2. Prepare data with only these features from the original DataFrame
X_train_sel = X_train[selected_features]
X_test_proc_sel = X_test[selected_features]

# 3. Identify column types for proper preprocessing from the DataFrame
cat_cols = X_train_sel.select_dtypes(include='object').columns.tolist()
num_cols = X_train_sel.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 4. Create preprocessing transformer for selected features
preprocessor_shap = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
])

# 5. Set class weight for imbalance (as in previous pipeline)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# 6. Create the full pipeline: preprocessing + SMOTE + classifier
pipeline_shap = ImbPipeline([
    ('preprocessor', preprocessor_shap),
    ('smote', SMOTE(random_state=42)),
    ('clf', XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight,
        tree_method='hist'
    ))
])
# 7. Fit the pipeline and evaluate on test data using the DataFrame with selected features
pipeline_shap.fit(X_train_sel, y_train)
y_pred_shap = pipeline_shap.predict(X_test_proc_sel)
y_proba_shap = pipeline_shap.predict_proba(X_test_proc_sel)[:, 1]

print(selected_features)
print(classification_report(y_test, y_pred_shap))
print(f"Accuracy:  {accuracy_score(y_test, y_pred_shap):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_shap):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_shap):.4f}")
print(f"F1-score:  {f1_score(y_test, y_pred_shap):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_proba_shap):.4f}")

In [ ]:

# Create a new pipeline that uses only the most important features from the best model.
# Here, we select the top 3 features: 'DEBTINC', 'DELINQ', and 'CLAGE'


selected_features = ['DEBTINC', 'DELINQ', 'CLAGE']

# Create a preprocessor that scales only the selected features
preprocessor_sel = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_features)
    ]
)

# Retrieve the best model's classifier from the manager pipeline
# (The best pipeline already contains a preprocessor and SMOTE, but here we rebuild the pipeline to work on only the selected features.)
best_clf = mgr.best_estimators_[best_model_name].named_steps['clf']

# Build a new pipeline using the selected features
pipeline_sel = ImbPipeline([
    ('preprocessor', preprocessor_sel),
    ('smote', SMOTE(random_state=42)),
    ('clf', best_clf)
])

# Fit the new pipeline on training data using only the selected features
pipeline_sel.fit(X_train[selected_features], y_train)

# Evaluate on the test set (using the same selected features)
y_pred_sel = pipeline_sel.predict(X_test[selected_features])
y_proba_sel = pipeline_sel.predict_proba(X_test[selected_features])[:, 1]

print("Performance of the best model on selected features:")
print(classification_report(y_test, y_pred_sel))
print("ROC AUC:", roc_auc_score(y_test, y_proba_sel))

### Observations on the Classification Report and Selected Features

- **Class-Specific Performance:**
    - **Non-Default Class (0):**
        - **Precision:** 0.95 indicates that when the model predicts a non-default, it is highly accurate.
        - **Recall:** 0.89 shows that most non-default cases are correctly identified.
        - **F1-Score:** 0.92 confirms strong overall performance for class 0.
    - **Default Class (1):**
        - **Precision:** 0.64 suggests that a notable proportion of non-defaulter predictions are incorrect when predicting a default.
        - **Recall:** 0.82 implies that the model is effective at capturing the majority of actual defaulters.
        - **F1-Score:** 0.72 represents a reasonable balance between precision and recall for this minority class.

- **Overall Metrics:**
    - **Accuracy:** 87% accuracy on the test set.
    - **Macro Average:** A precision of 0.80, recall of 0.85, and F1-score of 0.82, reflecting balanced performance across both classes.
    - **Weighted Average:** Metrics (precision = 0.89, recall = 0.87, F1-score = 0.88) are influenced by the larger non-default class and support overall model robustness.

- **Insights from Feature Selection and Pipeline:**
    - The pipeline was re-trained using a subset of top features (_e.g._, `DEBTINC`, `CLAGE`, `DELINQ`, `YOJ`, `CLNO`) after SHAP-based selection.
    - The choice of a limited yet influential feature set can simplify the model while retaining competitive predictive performance.
    - The use of SMOTE in the pipeline helps mitigate class imbalance and contributes to the higher recall for the default class.

- **Business Implications:**
    - The high recall for the default class (0.82) is critical in risk-sensitive applications like loan default prediction, as it minimizes missed high-risk applicants.
    - However, lower precision for the default class (0.64) indicates some false alarms, which might lead to some good applicants being flagged. Further calibration of decision thresholds could help balance this trade-off.


# Final Observations

**Key ROC Curve Observations:**

1. **Ensemble Models Performance:**  
    The ROC curves for the ensemble models (Random Forest and XGBoost) quickly rise towards a high true positive rate while keeping the false positive rate low. This indicates that at lower FPR thresholds, both models capture a high percentage of defaulters.

2. **High ROC-AUC Scores:**  
    The Random Forest model achieves an ROC-AUC of around 0.96, and the XGBoost model has an ROC-AUC of approximately 0.96. These high values demonstrate excellent discrimination ability between defaulters and non‑defaulters, especially when compared to simpler baseline models like Logistic Regression.

3. **Sensitivity at Low False Positive Rates:**  
    The steep initial portion of the ROC curves signifies that even when the false positive rate is minimal, the models are still able to detect a significant portion of positives (defaulters). This sensitivity is crucial in risk-sensitive applications, where failing to detect a defaulter can be costly.

4. **Advantage of Ensemble Methods:**  
    The overall shape of the ROC curves confirms that ensemble methods, by capturing non-linear patterns in the data, are a better fit for this dataset relative to simpler models. They significantly reduce the number of false negatives.

**Summary:**  
- Ensemble models (Random Forest and XGBoost) yield ROC-AUC scores of approximately 0.96 and 0.96, respectively.  
- The steep climb observed in the ROC curves demonstrates high sensitivity with a low rate of false positives.  
- Together, these observations validate that ensemble approaches are highly effective at distinguishing between defaulters and non‑defaulters.


### Observations from the Confusion Matrix Comparisons

- **Logistic Regression:**  
    - The confusion matrix shows that almost all test instances are classified as the non-default class.
    - This heavy bias toward the negative class indicates a very high number of false negatives (i.e. defaulters being missed).
    - High specificity comes at the cost of very low recall, making the model unsuitable for detecting defaulters.

- **Tuned Random Forest:**  
    - The confusion matrix for the tuned Random Forest exhibits a better balance by correctly identifying a higher number of defaulters.
    - Fewer false negatives are observed compared to Logistic Regression, indicating improved sensitivity.
    - This model seems to balance precision and recall more effectively—important for risk-sensitive applications.

- **Tuned XGBoost:**  
    - Similar to Random Forest, the confusion matrix for XGBoost shows a low number of false negatives.
    - The overall distribution of true positives and true negatives is competitive with the Random Forest, with only marginal differences.
    - Despite slight variations, XGBoost still captures the minority class better than Logistic Regression.

**Summary:**  
Ensemble methods (Random Forest and XGBoost) clearly outperform Logistic Regression by reducing the number of false negatives. This makes them more reliable for loan default prediction, as they are more effective at identifying high-risk applicants.

**Refined Insights:**
- Default risk is strongly influenced by delinquency counts (DELINQ) and credit history measures (CLAGE, DEROG), suggesting that previous credit trouble is a key indictor of future defaults.
- Loan and mortgage amounts (LOAN, MORTDUE) show wide ranges, hinting that segmentation or transformation might further improve risk evaluation.
- Different loan purposes (REASON) exhibit distinct default rates, with 'HomeImp' potentially carrying a slightly higher risk than 'DebtCon'.
- Ensemble models (Random Forest and XGBoost) clearly outperform Logistic Regression, demonstrating that complex interactions in the data are best captured using these methods.
- SHAP analyses confirm the dominance of features like DELINQ, CLAGE, and DEBTINC, reinforcing their importance in informing both risk models and business decisions.



## Recommendations
1. **Deep Dive on Influential Features:**  
    Explore further the features with high SHAP absolute values (e.g., DELINQ, CLAGE, DEBTINC) to derive domain-specific insights that could refine risk evaluation strategies.
2. **Model Decision Thresholds:**  
    Given the strong ROC-AUC and Precision-Recall metrics, consider adjusting classification thresholds to optimize false negative rates, particularly considering the cost associated with misclassifying defaulters.
3. **Communication and Transparency:**  
    Use the SHAP summary and feature importance plots to clearly convey how different features contribute to predictions in stakeholder communications.
4. **Further Exploration of Subsets:**  
    Validate the stability of SHAP value distributions by examining different sample subsets or through cross-validation, ensuring that the model’s interpretability holds across various scenarios.
5. **Continuous Monitoring and Refinement:**  
    Compare the performance of the current tuned ensemble models (Random Forest and XGBoost) with the baseline logistic regression to monitor performance over time and identify any need for refinement in preprocessing or model tuning.

These observations and recommendations support a data-driven approach in enhancing model understanding and risk management.


## **Executive Summary**

The loan default risk modeling project addressed the critical need to identify high-risk borrowers using data-driven machine learning techniques.

**Key findings from Milestone 1:**
- The dataset (HMEQ) was imbalanced, with default (BAD=1) being the minority class.
- Data preprocessing included outlier treatment, missing value imputation, encoding categorical variables, and scaling numerics.
- Exploratory analysis revealed that variables such as debt-to-income ratio (`DEBTINC`), number of delinquent lines (`DELINQ`), and age of oldest credit line (`CLAGE`) are strong predictors of default.

**Final proposed model:**
- The best-performing model is an XGBoost classifier, tuned using GridSearchCV and balanced with SMOTE and class weighting.
- After SHAP-based feature importance analysis, a simplified model using the top 4-5 most important features achieved almost identical performance to the full-feature model, enabling easier regulatory compliance and business transparency.

**Next steps:**
- Deploy the final XGBoost pipeline in the loan approval process.
- Integrate SHAP explanations for every decision to support explainability and compliance.
- Monitor model performance over time and periodically retrain on new data.
- Continue analyzing customer segments and edge cases to further reduce risk and optimize thresholds.

---

## **Problem and Solution Summary**

**Problem summary:**
- The business challenge is to **reduce financial losses** from loan defaults while maintaining high approval rates for good customers.
- Predicting defaults accurately is complicated by the imbalanced nature of historical defaults and the need for transparent, auditable decisions.

**Proposed solution design:**
- The solution is an end-to-end machine learning pipeline using XGBoost, enhanced by SMOTE balancing and thorough feature engineering.
- SHAP-based explainability is incorporated to make every risk prediction transparent for auditors and decision-makers.
- The final model can operate with as few as four key variables (`DEBTINC`, `CLAGE`, `DELINQ`, `LOAN`), without significant loss of predictive power.

**Business impact:**
- Enables the bank to identify high-risk applications more accurately, reducing expected losses.
- Increases trust and transparency for both regulators and internal stakeholders.
- Streamlines approval processes for low-risk customers, supporting business growth.

---

## **Recommendations for Implementation**

**Key recommendations:**
- **Productionize** the final XGBoost model as part of the automated loan approval workflow.
- Use SHAP explanations to provide individualized reason codes for loan decisions.
- Set up regular performance monitoring and schedule retraining every 3–6 months.
- Ensure data pipelines remain robust, with alerts for data drift or quality issues.

**Key actionables for stakeholders:**
- Credit/risk team: Review and approve the simplified scoring model.
- IT: Integrate the Python pipeline and SHAP reporting into existing systems.
- Compliance: Audit model explanations and monitor for bias or unfair impact.

**Expected benefits:**
- Reduction in default-related financial losses (potentially 10–20% annual savings based on historic portfolio).
- Improved efficiency and turnaround time for loan applications.
- Enhanced regulatory compliance, reducing audit risk and potential fines.

**Estimated costs and numbers (example rational assumption):**
- Implementation and integration cost: ~$50k (model development, IT integration, training).
- Expected annual benefit: For a $10M portfolio, a 1% reduction in default rate saves $100,000 per year.

**Key risks and challenges:**
- **Model drift** if customer behavior changes — mitigated by monitoring and retraining.
- **Data quality issues** (e.g., missing or delayed variables) could impact predictions.
- **Regulatory changes** may require updating the feature set or model logic.

**Further analysis/future work:**
- Deepen analysis of high-risk subsegments for tailored interventions.
- Explore alternative algorithms (e.g., LightGBM, explainable boosting machines) for continuous improvement.
- Consider deployment of dual models (full and minimal feature set) for different approval scenarios.



In [ ]:
import nbformat
from nbconvert import HTMLExporter

# Define the notebook file name
notebook_filename = "Applied_Data_Science_Loan_Default_Zuzanna_Walus.ipynb"

# Load the notebook content
with open(notebook_filename, 'r', encoding='utf-8') as f:
    notebook_content = nbformat.read(f, as_version=4)

# Convert the notebook to HTML
html_exporter = HTMLExporter()
html_data, _ = html_exporter.from_notebook_node(notebook_content)

# Save the HTML file
output_filename = "Applied_Data_Science_Loan_Default_Zuzanna_Walus.ipynb.html"
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(html_data)

print(f"Notebook successfully exported to {output_filename}")